In [1]:
from collections import namedtuple
import os


In [2]:
os.chdir("../")

In [3]:
from collections import namedtuple

# Define a named tuple type called 'Person' with fields 'name' and 'age'
Person = namedtuple('Person', ['name', 'age'])

# Create an instance of the named tuple
person = Person(name='John', age=30)

# Access fields using dot notation
print(person.name)  # Output: John
print(person.age)   # Output: 30


John
30


In [4]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
# By entity, we mean this 

In [6]:
from cnnClassifier.utils import read_yaml , create_directories
from cnnClassifier.constants import  *



class ConfigurationManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config_file = read_yaml(config_file_path)
        self.params_file = read_yaml(params_file_path)
        create_directories([self.config_file.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config_file.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
         if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url  = self.config.source_URL,
                filename = self.config.local_data_file
            )
    # to take only jpgs
    # These are the hidden function, user can access it
    def _get_update_list_file(self, list_of_file):
        return  [file for file in list_of_file if file.endswith(".jpg") and ("Cat" in file or "Dog" in file)]
    
    # to remove 0 kb files
    def _preprocess(self, zf, f, unzip_dir):
        target_file = os.path.join(unzip_dir, f)
        if not os.path.exists(target_file):
            zf.extract(f , unzip_dir)  # f: PetImages/Cat/0.jpg , unzip_dir: artifacts/data_ingestion, zf: zip file
        
        if os.path.getsize(target_file) == 0:
            os.remove(target_file)


    def unzip_and_clean(self):
        with ZipFile(self.config.local_data_file, 'r') as zf:
            list_of_file = zf.namelist()
            # cat/dog folder
            # This update return files as ['PetImages/Cat/0.jpg',...]
            update_list_file = self._get_update_list_file(list_of_file)    # ['PetImages/Cat/0.jpg',...] 
            
            # to take only the jpg files
            # This is the hidden function, user can't access it
            # Preprocess and extract single file and check whether the size is 0 or not
            for f in update_list_file:
                self._preprocess(zf, f ,self.config.unzip_dir)

In [13]:
'''
 

'''

'\n \n\n'

In [17]:
# with ZipFile("C:/Users/Pc/Downloads/Compressed/cat-dog-data.zip", 'r') as zf:
#             list_of_file = zf.namelist()

#             print([file for file in list_of_file if file.endswith(".jpg") and ("Cat" in file or "Dog" in file)])

In [15]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    print(e)




[2023-12-23 20:06:55,970: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-23 20:06:55,973: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-23 20:06:55,975: INFO: common: created directory at: artifacts]
[2023-12-23 20:06:55,976: INFO: common: created directory at: artifacts/data_ingestion]
